In [1]:
from transformers import pipeline

In [5]:
model_name = "distilbert-base-uncased" # a small version of BERT

nlp = pipeline("fill-mask", model_name) # create pipeline

In [20]:
#BAD EXAMPLE
sentence_to_classify = "perfect"

prompt = f"""
love => positive
hate => negative
great => positive
terrible => negative
{sentence_to_classify} => {nlp.tokenizer.mask_token}
"""

print(prompt)
print(nlp(prompt))


love => positive
hate => negative
great => positive
terrible => negative
perfect => [MASK]

[{'sequence': 'love = > positive hate = > negative great = > positive terrible = > negative perfect = > |', 'score': 0.25564244389533997, 'token': 1064, 'token_str': '|'}, {'sequence': 'love = > positive hate = > negative great = > positive terrible = > negative perfect = >.', 'score': 0.17725659906864166, 'token': 1012, 'token_str': '.'}, {'sequence': 'love = > positive hate = > negative great = > positive terrible = > negative perfect = > ;', 'score': 0.09742584079504013, 'token': 1025, 'token_str': ';'}, {'sequence': 'love = > positive hate = > negative great = > positive terrible = > negative perfect = > >', 'score': 0.05383085831999779, 'token': 1028, 'token_str': '>'}, {'sequence': 'love = > positive hate = > negative great = > positive terrible = > negative perfect = > negative', 'score': 0.04975255951285362, 'token': 4997, 'token_str': 'negative'}]


In [14]:
# examples
sentence_to_classify = "good"

prompt = f"""
beautiful = is a positive word
ugly = is a negative word
great = is a positive word
terrible = is a negative word
amazing = is a positive word
{sentence_to_classify} = is a {tokenizer.mask_token} word
"""

print(prompt)
print(nlp(prompt))


beautiful = is a positive word
ugly = is a negative word
great = is a positive word
terrible = is a negative word
amazing = is a positive word
good = is a [MASK] word

[{'sequence': 'beautiful = is a positive word ugly = is a negative word great = is a positive word terrible = is a negative word amazing = is a positive word good = is a negative word', 'score': 0.66949462890625, 'token': 4997, 'token_str': 'negative'}, {'sequence': 'beautiful = is a positive word ugly = is a negative word great = is a positive word terrible = is a negative word amazing = is a positive word good = is a positive word', 'score': 0.323766827583313, 'token': 3893, 'token_str': 'positive'}, {'sequence': 'beautiful = is a positive word ugly = is a negative word great = is a positive word terrible = is a negative word amazing = is a positive word good = is a bad word', 'score': 0.0016469251131638885, 'token': 2919, 'token_str': 'bad'}, {'sequence': 'beautiful = is a positive word ugly = is a negative word grea

In [24]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

# download and load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")

# create prompt:
sentence_to_classify = "stupid"
prompt = f"""
terrible is a negative word
ugly is a negative word
great is a positive word
amazing is a positive word
beautiful is a positive word
{sentence_to_classify} is a {tokenizer.mask_token} word
"""

# tokenize the input
input = tokenizer.encode(prompt, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]  # record the index of the mask token

# forward pass through the model
token_logits = model(input).logits
token_logits.shape  # (batch_size, tokens, vocabulary) in this case it is (1, 30, 30522)

# extract the most likely word for the MASK
mask_token_logits = token_logits[0, mask_token_index, :]  # select the mask token
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(tokenizer.decode([token]))

negative
positive
bad
good
terrible
